Web Scraping de SIEA

# Importacion paquetes

In [2]:
import os #Definir el directorio de trabajo
os.chdir(r'D:\Estudios_extra\Big_Data_Python\Aplicando_WEB_SCRAPING')
os.getcwd() #conocer el directorio de trabajo

'D:\\Estudios_extra\\Big_Data_Python\\Aplicando_WEB_SCRAPING'

### Paquetes

In [4]:
# pip install webdriver-manager
# pip install selenium
# pip install chromedriver-py

Librerias

In [3]:
from selenium import webdriver
from chromedriver_py import binary_path # this will get you the path variable
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import random

## Conexion a WEB SIEA

Aqui primero queremos conocer como están escritos todos los departamentos, por eso escojimos el cultivo que existe en todo el Perú.
> Esto nos servirá para darle la lista a Python y que busque esta lista en todos los cultivos.

In [4]:
try:
    svc = webdriver.ChromeService(executable_path=binary_path)
    driver = webdriver.Chrome(service=svc)
    url = "https://siea.midagri.gob.pe/portal/calendario/#"
    driver.get(url)
    
    # Esperar a que la página cargue
    time.sleep(5)
    
    # Seleccionar Maiz Amarillo Duro
    dropdown = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "span.select2-selection.select2-selection--single"))
    )
    dropdown.click()
    
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input.select2-search__field"))
    )
    search_box.clear()
    search_box.send_keys("Maiz Amarillo Duro")
    time.sleep(1)
    search_box.send_keys(Keys.RETURN)
    
    # Hacer click en el botón "cosecha" usando el ID correcto
    cosecha_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "btnCosecha"))
    )
    cosecha_button.click()
    time.sleep(2)

    # Crear objeto ActionChains para el hover
    action = ActionChains(driver)
    
    # Conjunto para almacenar departamentos únicos
    departamentos_encontrados = set()
    
    # Lista de todos los departamentos que deberíamos encontrar
    todos_departamentos = {
        'Amazonas', 'Áncash', 'Apurímac', 'Arequipa', 'Ayacucho', 'Cajamarca',
        'Cusco', 'Huancavelica', 'Huánuco', 'Ica', 'Junín', 'La Libertad',
        'Lambayeque', 'Lima', 'Loreto', 'Madre De Dios', 'Moquegua', 'Pasco',
        'Piura', 'Puno', 'San Martín', 'Tacna', 'Tumbes', 'Ucayali'
    }
    
    print("Identificando departamentos...")
    print("-" * 30)
    
    max_intentos = 1000
    intento = 0
    
    while len(departamentos_encontrados) < 24 and intento < max_intentos:
        # Encontrar todos los elementos del mapa
        departments = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "path.highcharts-point"))
        )
        
        # Seleccionar un elemento aleatorio para mayor cobertura
        for department in random.sample(departments, len(departments)):
            try:
                # Mover el cursor sobre el elemento
                action.move_to_element(department).perform()
                time.sleep(0.2)
                
                # Obtener el tooltip
                tooltip = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".highcharts-tooltip"))
                )
                
                # Extraer el nombre del departamento
                nombre_departamento = tooltip.text.strip()
                
                # Agregar al conjunto si es nuevo
                if nombre_departamento not in departamentos_encontrados:
                    departamentos_encontrados.add(nombre_departamento)
                    print(f"Departamento encontrado: {nombre_departamento}")
                    
                    # Mostrar departamentos faltantes
                    faltantes = todos_departamentos - departamentos_encontrados
                    if faltantes:
                        print(f"Faltan: {sorted(faltantes)}")
                        print(f"Progreso: {len(departamentos_encontrados)}/24")
                        print("-" * 30)
                    
            except Exception as e:
                continue
        
        intento += 1
    
    # Imprimir resumen final
    print("\nResumen final:")
    print(f"Total de departamentos encontrados: {len(departamentos_encontrados)}")
    print("\nLista completa de departamentos encontrados:")
    for i, dept in enumerate(sorted(departamentos_encontrados), 1):
        print(f"{i}. {dept}")
    
    # Verificar si faltó algún departamento
    faltantes = todos_departamentos - departamentos_encontrados
    if faltantes:
        print("\nDepartamentos no encontrados:")
        for dept in sorted(faltantes):
            print(f"- {dept}")

except Exception as e:
    print(f"Error general: {str(e)}")
finally:
    time.sleep(2)
    print("driver.quit()")

Error general: Message: session not created: This version of ChromeDriver only supports Chrome version 131
Current browser version is 135.0.7049.85 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
	GetHandleVerifier [0x00007FF6480980D5+2992373]
	(No symbol) [0x00007FF647D2BFD0]
	(No symbol) [0x00007FF647BC590A]
	(No symbol) [0x00007FF647C07662]
	(No symbol) [0x00007FF647C066CB]
	(No symbol) [0x00007FF647C009E3]
	(No symbol) [0x00007FF647BFBC39]
	(No symbol) [0x00007FF647C4A6AC]
	(No symbol) [0x00007FF647C49C90]
	(No symbol) [0x00007FF647C3F113]
	(No symbol) [0x00007FF647C0A918]
	(No symbol) [0x00007FF647C0BA81]
	GetHandleVerifier [0x00007FF6480F6A2D+3379789]
	GetHandleVerifier [0x00007FF64810C32D+3468109]
	GetHandleVerifier [0x00007FF648100043+3418211]
	GetHandleVerifier [0x00007FF647E8C78B+847787]
	(No symbol) [0x00007FF647D3757F]
	(No symbol) [0x00007FF647D32FC4]
	(No symbol) [0x00007FF647D3315D]
	(No symbol) [0x00007FF647D22979]
	BaseThreadInitThunk

### Nuevo intento en cuadricula

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import random

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# Configuración correcta para las versiones recientes
options = Options()
service = Service(ChromeDriverManager().install())

# Inicializar el driver
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://siea.midagri.gob.pe/portal/calendario/#")

In [79]:
# Seleccionar Maiz Amarillo Duro
dropdown = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "span.select2-selection.select2-selection--single"))
)
dropdown.click()

search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "input.select2-search__field"))
)
search_box.clear()
search_box.send_keys("Maiz Amarillo Duro")
time.sleep(1)
search_box.send_keys(Keys.RETURN)

# Hacer click en el botón "cosecha" usando el ID correcto
cosecha_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "btnCosecha"))
)
cosecha_button.click()
time.sleep(2)

In [75]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import re

# Conjunto para almacenar departamentos encontrados
departamentos_encontrados = set()
todos_departamentos = {
    'Amazonas', 'Áncash', 'Apurímac', 'Arequipa', 'Ayacucho', 'Cajamarca',
    'Cusco', 'Huancavelica', 'Huánuco', 'Ica', 'Junín', 'La Libertad',
    'Lambayeque', 'Lima', 'Loreto', 'Madre De Dios', 'Moquegua', 'Pasco',
    'Piura', 'Puno', 'San Martín', 'Tacna', 'Tumbes', 'Ucayali'
}

print("Iniciando exploración visual del mapa...")

try:
    # Definir las coordenadas específicas de la zona del mapa (lo que ves en la imagen)
    # Estas coordenadas están ajustadas según lo que se ve en la captura de pantalla
    x_min = 150  # Borde izquierdo del mapa
    y_min = 200  # Borde superior del mapa
    x_max = 500  # Borde derecho del mapa
    y_max = 650  # Borde inferior del mapa
    
    # Definir una cuadrícula 10x10
    grid_size = 10
    cell_width = (x_max - x_min) / grid_size
    cell_height = (y_max - y_min) / grid_size
    
    # Crear ActionChains para mover el cursor
    action = ActionChains(driver)
    
    # Limpiar marcadores previos y dibujar contorno
    driver.execute_script("""
        // Limpiar marcadores previos
        document.querySelectorAll('div[style*="red"]').forEach(el => el.remove());
        
        // Añadir contorno del área
        var outline = document.createElement('div');
        outline.style.position = 'absolute';
        outline.style.left = '150px';
        outline.style.top = '200px';
        outline.style.width = '350px';
        outline.style.height = '450px';
        outline.style.border = '2px solid blue';
        outline.style.zIndex = '9999';
        outline.style.pointerEvents = 'none';
        document.body.appendChild(outline);
    """)
    
    # Explorar el mapa celda por celda
    print("Moviendo el cursor a través del mapa...")
    
    for row in range(grid_size):
        for col in range(grid_size):
            # Calcular el centro de esta celda
            center_x = x_min + (col + 0.5) * cell_width
            center_y = y_min + (row + 0.5) * cell_height
            
            # Imprimir información para depuración
            print(f"Moviendo a: X={center_x}, Y={center_y} (celda [{row},{col}])")
            
            # Usar JavaScript para mover el cursor a esta posición
            script = f
                var evt = new MouseEvent('mousemove', {{
                    'view': window,
                    'bubbles': true,
                    'cancelable': true,
                    'clientX': {center_x},
                    'clientY': {center_y}
                }});
                
                var element = document.elementFromPoint({center_x}, {center_y});
                if (element) {{
                    element.dispatchEvent(evt);
                    // Resaltar el punto para visualización
                    var marker = document.createElement('div');
                    marker.style.position = 'absolute';
                    marker.style.left = '{center_x}px';
                    marker.style.top = '{center_y}px';
                    marker.style.width = '8px';
                    marker.style.height = '8px';
                    marker.style.backgroundColor = 'red';
                    marker.style.borderRadius = '50%';
                    marker.style.zIndex = '10000';
                    document.body.appendChild(marker);
                }}
                
                // Ver si hay un tooltip visible y obtener su texto
                var tooltipText = '';
                var tooltips = document.querySelectorAll('.highcharts-tooltip, g.highcharts-tooltip, .highcharts-tooltip-container, [class*="tooltip"], text, tspan');
                for(var i = 0; i < tooltips.length; i++) {{
                    if(tooltips[i].style && tooltips[i].style.visibility !== 'hidden' && 
                       tooltips[i].style.display !== 'none' && 
                       (tooltips[i].textContent || '').trim() !== '') {{
                        tooltipText = tooltips[i].textContent.trim();
                        break;
                    }}
                }}
                return tooltipText;
           
            
            # Ejecutar el script y obtener posible texto del tooltip
            tooltip_text = driver.execute_script(script)
            
            # Esperar para dar tiempo a que aparezca el tooltip
            time.sleep(2)  # Aumentamos el tiempo de espera
            
            # Si no obtenemos texto con JavaScript, intentar con métodos de Selenium
            if not tooltip_text:
                try:
                    # Método 1: Buscar por CSS selector (más específicos)
                    tooltip_selectors = [
                        ".highcharts-tooltip", 
                        "g.highcharts-tooltip", 
                        ".highcharts-tooltip-container",
                        "[class*='tooltip']",
                        "text.highcharts-text",
                        "tspan",
                        "g text"
                    ]
                    
                    for selector in tooltip_selectors:
                        elements = driver.find_elements(By.CSS_SELECTOR, selector)
                        for element in elements:
                            if element.is_displayed():
                                elemento_texto = element.text.strip()
                                if elemento_texto:
                                    tooltip_text = elemento_texto
                                    print(f"  Tooltip encontrado con selector '{selector}': {tooltip_text}")
                                    break
                        if tooltip_text:
                            break
                    
                    # Método 2: Si fallaron los selectores, obtener todos los textos visibles
                    if not tooltip_text:
                        visible_texts = driver.find_elements(By.XPATH, "//text[contains(., '.')]")
                        for text in visible_texts:
                            texto = text.text.strip()
                            if texto and any(depto in texto for depto in todos_departamentos):
                                tooltip_text = texto
                                print(f"  Texto departamento encontrado: {tooltip_text}")
                                break
                
                except Exception as e:
                    print(f"  Error buscando tooltip: {e}")
            
            # Si encontramos texto de tooltip, buscar departamentos
            if tooltip_text:
                print(f"  → Tooltip encontrado: {tooltip_text}")
                
                # Extraer departamento del texto del tooltip
                for departamento in todos_departamentos:
                    # Buscar el departamento en el texto
                    if departamento in tooltip_text and departamento not in departamentos_encontrados:
                        departamentos_encontrados.add(departamento)
                        print(f"➕ ENCONTRADO: {departamento}")
                        print(f"   Progreso: {len(departamentos_encontrados)}/24")
                        break
                
                # Si no encontramos coincidencia directa pero hay porcentaje, probar extracción
                if "%" in tooltip_text and len(departamentos_encontrados) == 0:
                    # Extraer texto antes del porcentaje (suele ser el nombre)
                    match = re.search(r'([^:0-9%]+)[:0-9\s\.%]+', tooltip_text)
                    if match:
                        nombre_extraido = match.group(1).strip()
                        print(f"  Nombre extraído del tooltip: '{nombre_extraido}'")
                        
                        # Buscar coincidencia aproximada
                        for departamento in todos_departamentos:
                            if departamento.lower() in nombre_extraido.lower() or nombre_extraido.lower() in departamento.lower():
                                if departamento not in departamentos_encontrados:
                                    departamentos_encontrados.add(departamento)
                                    print(f"➕ ENCONTRADO (coincidencia aproximada): {departamento}")
                                    print(f"   Progreso: {len(departamentos_encontrados)}/24")
                                    break
    
    # Mostrar resultados
    print("\n===== RESULTADOS FINALES =====")
    print(f"Total de departamentos encontrados: {len(departamentos_encontrados)}")
    
    if departamentos_encontrados:
        print("\nDepartamentos encontrados:")
        for i, dept in enumerate(sorted(departamentos_encontrados), 1):
            print(f"{i}. {dept}")
    
    # Verificar faltantes
    faltantes = todos_departamentos - departamentos_encontrados
    if faltantes:
        print("\nDepartamentos no encontrados:")
        for dept in sorted(faltantes):
            print(f"- {dept}")
    
except Exception as e:
    print(f"Error general: {str(e)}")

Iniciando exploración visual del mapa...
Moviendo el cursor a través del mapa...
Moviendo a: X=167.5, Y=222.5 (celda [0,0])
  → Tooltip encontrado: Perú
Moviendo a: X=202.5, Y=222.5 (celda [0,1])
  → Tooltip encontrado: Perú
Moviendo a: X=237.5, Y=222.5 (celda [0,2])
  → Tooltip encontrado: Perú
Moviendo a: X=272.5, Y=222.5 (celda [0,3])
  → Tooltip encontrado: Perú
Moviendo a: X=307.5, Y=222.5 (celda [0,4])
  → Tooltip encontrado: Perú
Moviendo a: X=342.5, Y=222.5 (celda [0,5])
  → Tooltip encontrado: Perú
Moviendo a: X=377.5, Y=222.5 (celda [0,6])
  → Tooltip encontrado: Perú
Moviendo a: X=412.5, Y=222.5 (celda [0,7])
  → Tooltip encontrado: Perú
Moviendo a: X=447.5, Y=222.5 (celda [0,8])
  → Tooltip encontrado: Perú
Moviendo a: X=482.5, Y=222.5 (celda [0,9])
  → Tooltip encontrado: Perú
Moviendo a: X=167.5, Y=267.5 (celda [1,0])
  → Tooltip encontrado: Perú
Moviendo a: X=202.5, Y=267.5 (celda [1,1])
  → Tooltip encontrado: Perú
Moviendo a: X=237.5, Y=267.5 (celda [1,2])
  → Tooltip 

Busca el departamento y hace click:

In [ ]:
try:
    # Configuración inicial
    time.sleep(2)
    action = ActionChains(driver)
    departments = driver.find_elements(By.CSS_SELECTOR, "path.highcharts-point")
    
    # Buscar Lima en los departamentos
    for department in departments:
        try:
            action.move_to_element(department).perform()
            tooltip = driver.find_element(By.CSS_SELECTOR, ".highcharts-tooltip")
            
            if "Moquegua" in tooltip.text.strip():
                # Intentar hacer clic usando diferentes métodos
                try:
                    department.click()
                except:
                    try:
                        action.click(department).perform()
                    except:
                        driver.execute_script("arguments[0].dispatchEvent(new MouseEvent('click', {bubbles: true}));", department)
                break
        except:
            continue
            
except Exception as e:
    print(f"Error: {e}")

Error: name 'driver' is not defined


In [13]:
    # Esperar 3 segundos después de hacer click en Moquegua
    time.sleep(4)

In [39]:
try:    
    # Buscar el botón Regresar usando el selector específico de SVG
    regresar_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".highcharts-button-box"))
    )
    
    # Usar ActionChains para mover el mouse y hacer click en Regresar
    action = ActionChains(driver)
    action.move_to_element(regresar_button).click().perform()
    
    print("✓ Clic realizado en botón Regresar")
    
    # Esperar un momento para que la página se actualice después de hacer clic en Regresar
    time.sleep(2)
    
    # Hacer click en el botón "Cosechas" usando el ID correcto
    try:
        # Primer intento: buscar por ID
        cosecha_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "btnCosecha"))
        )
        cosecha_button.click()
        print("✓ Clic realizado en botón Cosechas usando ID")
    except Exception as cosecha_error:
        print(f"Error al buscar por ID: {cosecha_error}")


except Exception as e:
    print(f"Error al hacer click en Regresar: {e}")

✓ Clic realizado en botón Regresar
✓ Clic realizado en botón Cosechas usando ID


In [ ]:
#ESTO ERA EL BOTON ANTERIOR DE "REGRESAR"

# try:    
#     # Buscar el botón Regresar usando el selector específico de SVG
#     regresar_button = WebDriverWait(driver, 10).until(
#         EC.element_to_be_clickable((By.CSS_SELECTOR, ".highcharts-button-box"))
#     )
    
#     # Usar ActionChains para mover el mouse y hacer click
#     action = ActionChains(driver)
#     action.move_to_element(regresar_button).click().perform()

# except Exception as e:
#     print(f"Error al hacer click en Regresar: {e}")

In [16]:
# ESTO ERA EL CODIGO PARA HACER CLICK EN DEPARTAMENTOS (EJ. MOQUEGUA)

# try:
#     # Encontrar elemento de Moquegua
#     departments = WebDriverWait(driver, 10).until(
#         EC.presence_of_all_elements_located((By.CSS_SELECTOR, "path.highcharts-point"))
#     )
#     for department in departments:
#         action.move_to_element(department).perform()
#         tooltip = WebDriverWait(driver, 2).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, ".highcharts-tooltip"))
#         )
#         if "Moquegua" in tooltip.text.strip():
#             action.move_to_element(department).perform()
#             department.click()
#             break
# except Exception as e:
#     print(f"Error al posicionar en Moquegua: {e}")

Prueba de 24 departamentos

In [33]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

# Lista de todos los departamentos del Perú
departamentos = [
    "Amazonas", "Áncash", "Apurímac", "Arequipa", "Ayacucho", 
    "Cajamarca", "Cusco", "Huancavelica", "Huánuco", "Ica", 
    "Junín", "La Libertad", "Lambayeque", "Lima", "Loreto", 
    "Madre De Dios", "Moquegua", "Pasco", "Piura", "Puno", 
    "San Martín", "Tacna", "Tumbes", "Ucayali"
]

# Función para hacer clic en un departamento específico
def click_departamento(nombre_departamento):
    try:
        print(f"Buscando {nombre_departamento}...")
        time.sleep(2)
        action = ActionChains(driver)
        departments = driver.find_elements(By.CSS_SELECTOR, "path.highcharts-point")
        
        for department in departments:
            try:
                action.move_to_element(department).perform()
                tooltip = driver.find_element(By.CSS_SELECTOR, ".highcharts-tooltip")
                
                if nombre_departamento in tooltip.text.strip():
                    print(f"¡{nombre_departamento} encontrado! Haciendo clic...")
                    try:
                        department.click()
                    except:
                        try:
                            action.click(department).perform()
                        except:
                            driver.execute_script("arguments[0].dispatchEvent(new MouseEvent('click', {bubbles: true}));", department)
                    return True
            except:
                continue
        
        print(f"{nombre_departamento} no encontrado")
        return False
                
    except Exception as e:
        print(f"Error al buscar {nombre_departamento}: {e}")
        return False

# Función para hacer clic en el botón Regresar
def click_regresar():
    try:
        print("Haciendo clic en Regresar...")
        time.sleep(2)  # Esperar a que se cargue la vista del departamento
        
        regresar_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".highcharts-button-box"))
        )
        
        action = ActionChains(driver)
        action.move_to_element(regresar_button).click().perform()
        
        # Esperar a que regrese a la vista principal
        time.sleep(2)
        return True
        
    except Exception as e:
        print(f"Error al hacer clic en Regresar: {e}")
        return False

# Función principal que itera por todos los departamentos
def explorar_departamentos():
    resultados = {}
    
    for departamento in departamentos:
        print(f"\n--- Procesando {departamento} ---")
        
        # Intentar hacer clic en el departamento
        if click_departamento(departamento):
            resultados[departamento] = "Clic exitoso"
            
            # Esperar un momento para ver los datos
            time.sleep(3)
            
            # Hacer clic en Regresar
            if not click_regresar():
                print(f"⚠️ No se pudo regresar después de {departamento}")
                resultados[departamento] += ", pero falló al regresar"
                
                # Intentar refrescar la página si no se puede regresar
                driver.refresh()
                time.sleep(5)
        else:
            resultados[departamento] = "Falló"
    
    # Resumen final
    print("\n\n--- RESUMEN DE RESULTADOS ---")
    for dep, result in resultados.items():
        print(f"{dep}: {result}")

# Ejecutar el script principal
explorar_departamentos()


--- Procesando Amazonas ---
Buscando Amazonas...
¡Amazonas encontrado! Haciendo clic...
Haciendo clic en Regresar...

--- Procesando Áncash ---
Buscando Áncash...
¡Áncash encontrado! Haciendo clic...
Haciendo clic en Regresar...

--- Procesando Apurímac ---
Buscando Apurímac...
¡Apurímac encontrado! Haciendo clic...
Haciendo clic en Regresar...

--- Procesando Arequipa ---
Buscando Arequipa...
¡Arequipa encontrado! Haciendo clic...
Haciendo clic en Regresar...

--- Procesando Ayacucho ---
Buscando Ayacucho...
¡Ayacucho encontrado! Haciendo clic...
Haciendo clic en Regresar...

--- Procesando Cajamarca ---
Buscando Cajamarca...
¡Cajamarca encontrado! Haciendo clic...
Haciendo clic en Regresar...

--- Procesando Cusco ---
Buscando Cusco...
¡Cusco encontrado! Haciendo clic...
Haciendo clic en Regresar...

--- Procesando Huancavelica ---
Buscando Huancavelica...
¡Huancavelica encontrado! Haciendo clic...
Haciendo clic en Regresar...

--- Procesando Huánuco ---
Buscando Huánuco...
¡Huánuco

## Buscando provincias

### Busqueda de coordenadas en pantalla

In [ ]:
#pip install pyautogui

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installi


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pyautogui

print("Mueve el cursor a cualquier lugar y presiona ENTER")
input()
x, y = pyautogui.position()
print(f"Posición del cursor: X={x}, Y={y}")

Mueve el cursor a cualquier lugar y presiona ENTER
Posición del cursor: X=1093, Y=512


In [10]:
import pyautogui

print("Mueve el cursor a cualquier lugar y presiona ENTER")
input()
x, y = pyautogui.position()
print(f"Posición del cursor: X={x}, Y={y}")

Mueve el cursor a cualquier lugar y presiona ENTER
Posición del cursor: X=812, Y=511


In [21]:
import pyautogui

print("Mueve el cursor a cualquier lugar y presiona ENTER")
input()
x, y = pyautogui.position()
print(f"Posición del cursor: X={x}, Y={y}")

Mueve el cursor a cualquier lugar y presiona ENTER
Posición del cursor: X=1011, Y=697


### Buscar provincia (robusta)

Buscando provincias de manera mas robusta por coordenadas, ya que tiene areas pequeñas.

In [17]:
def listar_provincias_mapa_mejorado(driver):
    """
    Función mejorada para detectar provincias con formas irregulares y en los bordes
    """
    try:
        action = ActionChains(driver)
        provincias_encontradas = set()
        
        print("\nIniciando barrido detallado del mapa...")
        print("-" * 40)
        
        # Puntos de offset ampliados para mejor cobertura
        # Incluye más puntos en los bordes y esquinas
        offsets = [
            (0, 0),     # Centro
            # Puntos cardinales
            (0, -10),   # Norte
            (0, 10),    # Sur
            (-10, 0),   # Oeste
            (10, 0),    # Este
            # Esquinas principales
            (-10, -10), # Noroeste
            (10, -10),  # Noreste
            (-10, 10),  # Suroeste
            (10, 10),   # Sureste
            # Puntos intermedios
            (-5, -5), (5, -5), (-5, 5), (5, 5),
            # Puntos adicionales para formas irregulares
            (-7, -7), (7, -7), (-7, 7), (7, 7),
            (-3, -3), (3, -3), (-3, 3), (3, 3),
            # Puntos extremos para bordes
            (-12, 0), (12, 0), (0, -12), (0, 12),
            # Puntos diagonales extendidos
            (-12, -12), (12, -12), (-12, 12), (12, 12)
        ]
        
        max_intentos = 4  # Aumentamos los intentos
        for intento in range(max_intentos):
            elementos_mapa = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "path.highcharts-point"))
            )
            
            print(f"\nIntento {intento + 1} - Elementos encontrados: {len(elementos_mapa)}")
            
            # Diferentes patrones de recorrido en cada intento
            if intento == 0:
                elementos = elementos_mapa  # Normal
            elif intento == 1:
                elementos = reversed(elementos_mapa)  # Reverso
            elif intento == 2:
                elementos = random.sample(elementos_mapa, len(elementos_mapa))  # Aleatorio
            else:
                # Patrón en espiral desde el centro
                centro = len(elementos_mapa) // 2
                elementos = elementos_mapa[centro:] + elementos_mapa[:centro]
            
            for elemento in elementos:
                # Obtener el tamaño y posición del elemento
                try:
                    size = elemento.size
                    location = elemento.location
                    
                    # Ajustar offsets según el tamaño del elemento
                    elemento_offsets = offsets.copy()
                    if size['width'] < 20 or size['height'] < 20:
                        # Agregar puntos adicionales para elementos pequeños
                        elemento_offsets.extend([
                            (-2, -2), (2, -2), (-2, 2), (2, 2),
                            (-1, -1), (1, -1), (-1, 1), (1, 1)
                        ])
                    
                    for offset_x, offset_y in elemento_offsets:
                        try:
                            # Movimiento más suave
                            action.move_to_element(elemento)\
                                  .pause(0.1)\
                                  .move_by_offset(offset_x, offset_y)\
                                  .pause(0.1)\
                                  .perform()
                            
                            # Intentar obtener el tooltip
                            tooltip = WebDriverWait(driver, 1).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, ".highcharts-tooltip"))
                            )
                            
                            nombre_provincia = tooltip.text.strip()
                            
                            if nombre_provincia and nombre_provincia not in provincias_encontradas:
                                provincias_encontradas.add(nombre_provincia)
                                print(f"¡Nueva provincia encontrada: {nombre_provincia}")
                                print(f"Posición: {location}")
                                print(f"Tamaño: {size}")
                                print(f"Offset usado: ({offset_x}, {offset_y})")
                                print(f"Total actual: {len(provincias_encontradas)}")
                                print("-" * 40)
                            
                            # Reset del cursor
                            action.move_to_element(elemento).perform()
                            
                        except Exception:
                            continue
                            
                except Exception as e:
                    print(f"Error al procesar elemento: {str(e)}")
                    continue
                
                # Pequeña pausa entre elementos
                time.sleep(0.1)
        
        # Imprimir resumen final
        print("\nResumen final de provincias encontradas:")
        print("-" * 40)
        for i, prov in enumerate(sorted(provincias_encontradas), 1):
            print(f"{i}. {prov}")
        print(f"\nTotal de provincias encontradas: {len(provincias_encontradas)}")
        
        return provincias_encontradas

    except Exception as e:
        print(f"Error general: {str(e)}")
        return set()

# Ejecutar la función
provincias_mapa = listar_provincias_mapa_mejorado(driver)


Iniciando barrido detallado del mapa...
----------------------------------------

Intento 1 - Elementos encontrados: 28
¡Nueva provincia encontrada: Lucanas
Posición: {'x': 649, 'y': 598}
Tamaño: {'height': 10.560791015625, 'width': 10.8057861328125}
Offset usado: (0, 0)
Total actual: 1
----------------------------------------
¡Nueva provincia encontrada: Huamanga
Posición: {'x': 534, 'y': 235}
Tamaño: {'height': 99.86390686035156, 'width': 160.3299560546875}
Offset usado: (0, 0)
Total actual: 2
----------------------------------------
¡Nueva provincia encontrada: Huanta
Posición: {'x': 554, 'y': 87}
Tamaño: {'height': 159.3710174560547, 'width': 135.3802490234375}
Offset usado: (0, 0)
Total actual: 3
----------------------------------------
¡Nueva provincia encontrada: La Mar
Posición: {'x': 633, 'y': 152}
Tamaño: {'height': 171.857177734375, 'width': 169.203125}
Offset usado: (0, 0)
Total actual: 4
----------------------------------------
¡Nueva provincia encontrada: Vilcas Huaman
P

In [47]:
print("\nContenido de la variable provincias:")
for i, provincias_mapa in enumerate(sorted(provincias_mapa), 1):
    print(f"{i}. {provincias_mapa}")


Contenido de la variable provincias:
1. Bellavista
2. El Dorado
3. Huallaga
4. Lamas
5. Mariscal Caceres
6. Moyobamba
7. Picota
8. Rioja
9. San Martin
10. Tocache
